In [ ]:
import tensorflow as tf
import json
# import keras
# from keras.layers import *
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
import tensorflow as tf
from tensorflow.python.keras.layers import *
from tensorflow.python import keras

import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
blog_posts_data_dir = "/content/drive/MyDrive/datasets/"
# blog_posts_data_dir = "/content/drive/MyDrive/datasets/BT-AP-19 Corpus/all/"
train_file_name = "train.json"
test_file_name = "test.json"

# Load data
with open(blog_posts_data_dir + train_file_name) as r:
    training_set = json.load(r)
training_set[0]

{'age': '16', 'gender': 'female', 'post': 'ooh shiny new commenting!'}

In [ ]:
raw_posts = [instance["post"] for instance in training_set]
len(raw_posts)

526812

In [ ]:
import numpy as np

EXAMINE = 21
SEED = 22
np.random.seed(SEED)

In [ ]:
def get_gender_as_num(gender):
    if gender == "male":
        return 0
    else:
        return 1

In [ ]:
def get_age_group(age): # HIGH NOTE: changing each of the scalars to a vector. This is probably not a good idea
    age = int(age)
    if age == 18:
        # 13 - 17
        return [1, 0, 0, 0]
    elif age == 25:
        # 23 - 27
        return [0, 1, 0, 0]
    elif age == 35:
        # 33 - 48
        return [0, 0, 1, 0]
    elif age == 50:
        # 33 - 48
        return [0, 0, 0, 1]
    else:
        return [0, 0, 0, 0]

In [ ]:
def get_age_group(age): # HIGH NOTE: changing each of the scalars to a vector. This is probably not a good idea
    if age < 18:
        # 13 - 17
        return [1, 0, 0]
    elif age < 28:
        # 23 - 27
        return [0, 1, 0]
    elif age < 49:
        # 33 - 48
        return [0, 0, 1]
    else:
        return [0, 0, 0]

In [ ]:
vocab_size = 20000  # Only consider the top 20k words
maxlen = 300  # Only consider the first 2

median_words_per_sample = maxlen
median_words_per_tokenized_sample = maxlen

# Map each word to a unique int value
MAX_WORD_COUNT = 20000
tokenizer = keras.preprocessing.text.Tokenizer(num_words = MAX_WORD_COUNT)
posts = [instance["post"] for instance in training_set]
tokenizer.fit_on_texts(posts)
word_index = dict(list(tokenizer.word_index.items())[:20000])
sequences = tokenizer.texts_to_sequences(posts)
# median_words_per_tokenized_sample = np.median([len(post) for post in sequences])
data = keras.preprocessing.sequence.pad_sequences(sequences, maxlen = maxlen,
                                                     padding = "post")
for i, instance in enumerate(training_set):
    instance["post"] = data[i]
    instance["gender"] = get_gender_as_num(instance["gender"])
    instance["age"] = get_age_group(int(instance["age"]))

## Age

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [ ]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

import os
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)
with strategy.scope():

    inputs = layers.Input(shape=(maxlen,))
    embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
    x = embedding_layer(inputs)
    transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
    x = transformer_block(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(20, activation="relu")(x)
    x = layers.Dropout(0.1)(x)
    outputs = layers.Dense(3, activation="softmax")(x)


    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile("adam", "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
posts_train = np.array([instance["post"] for instance in training_set])
ages_train = np.array([instance["age"] for instance in training_set])

In [ ]:
history = model.fit(
    posts_train, ages_train, batch_size=500, epochs=4, validation_split = 0.2
)

Epoch 1/4
843/843 [==============================] - 41s 39ms/step - loss: 0.9084 - accuracy: 0.5657 - val_loss: 0.7341 - val_accuracy: 0.6662
Epoch 2/4
843/843 [==============================] - 24s 29ms/step - loss: 0.7074 - accuracy: 0.6833 - val_loss: 0.7139 - val_accuracy: 0.6766
Epoch 3/4
843/843 [==============================] - 24s 29ms/step - loss: 0.6776 - accuracy: 0.6982 - val_loss: 0.7124 - val_accuracy: 0.6760
Epoch 4/4
843/843 [==============================] - 25s 29ms/step - loss: 0.6579 - accuracy: 0.7060 - val_loss: 0.7127 - val_accuracy: 0.6770


In [ ]:
# Load data
with open(blog_posts_data_dir + test_file_name) as r:
    test_set = json.load(r)

test_posts = [instance["post"] for instance in test_set]
test_sequences = tokenizer.texts_to_sequences(test_posts)
test_post_data = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen = int(median_words_per_tokenized_sample),
                                                     padding = "post")
for i, instance in enumerate(test_set):
    instance["post"] = test_post_data[i]
    instance["gender"] = get_gender_as_num(instance["gender"])
    instance["age"] = get_age_group(int(instance["age"]))

posts_test = np.array([instance["post"] for instance in test_set])
ages_test = np.array([instance["age"] for instance in test_set])

In [ ]:
model.evaluate(posts_test, ages_test)

4116/4116 [==============================] - 68s 16ms/step - loss: 0.7106 - accuracy: 0.6798


[0.7105690836906433, 0.6798326373100281]

In [ ]:
from sklearn.metrics import classification_report
pred = model.predict(posts_test)
print(classification_report(np.argmax(ages_test, axis=1), np.argmax(pred, axis=1)))

              precision    recall  f1-score   support

           0       0.80      0.66      0.72     44411
           1       0.64      0.82      0.72     62574
           2       0.60      0.36      0.45     24718

    accuracy                           0.68    131703
   macro avg       0.68      0.61      0.63    131703
weighted avg       0.69      0.68      0.67    131703



##Gender

In [ ]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

import os
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
strategy = tf.distribute.experimental.TPUStrategy(resolver)
with strategy.scope():

    inputs = layers.Input(shape=(maxlen,))
    embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
    x = embedding_layer(inputs)
    transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
    x = transformer_block(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(20, activation="relu")(x)
    x = layers.Dropout(0.1)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)


    model2 = keras.Model(inputs=inputs, outputs=outputs)
    model2.compile("adam", "binary_crossentropy", metrics=["accuracy"])

In [ ]:
posts_train = np.array([instance["post"] for instance in training_set])
genders_train = np.array([instance["gender"] for instance in training_set])

In [ ]:
history = model2.fit(
    posts_train, genders_train, batch_size=500, epochs=4, validation_split = 0.2
)

Epoch 1/4
843/843 [==============================] - 42s 40ms/step - loss: 0.6386 - accuracy: 0.6190 - val_loss: 0.5753 - val_accuracy: 0.6901
Epoch 2/4
843/843 [==============================] - 23s 28ms/step - loss: 0.5541 - accuracy: 0.7088 - val_loss: 0.5666 - val_accuracy: 0.6942
Epoch 3/4
843/843 [==============================] - 24s 29ms/step - loss: 0.5393 - accuracy: 0.7184 - val_loss: 0.5674 - val_accuracy: 0.6967
Epoch 4/4
843/843 [==============================] - 24s 29ms/step - loss: 0.5275 - accuracy: 0.7229 - val_loss: 0.5683 - val_accuracy: 0.6946


In [ ]:
posts_test = np.array([instance["post"] for instance in test_set])
genders_test = np.array([instance["gender"] for instance in test_set])

In [ ]:
model2.evaluate(posts_test, genders_test)

4116/4116 [==============================] - 69s 17ms/step - loss: 0.5695 - accuracy: 0.6951


[0.5694702863693237, 0.6951170563697815]

In [ ]:
from sklearn.metrics import classification_report
pred = model2.predict(posts_test)
print(classification_report(genders_test, (pred > 0.5).astype(int)))

              precision    recall  f1-score   support

           0       0.70      0.70      0.70     67312
           1       0.69      0.69      0.69     64391

    accuracy                           0.70    131703
   macro avg       0.69      0.69      0.69    131703
weighted avg       0.70      0.70      0.70    131703

